# 🚀 Phase 1: Environment Setup & Bronze Ingestion
**Project:** "Olist-Next" Hyper-Personalized Retention Engine
**Architecture:** Medallion Lakehouse (Bronze -> Silver -> Gold)
**Author:** Tanmay Patil

## 📋 Overview
This notebook establishes the foundation of the Lakehouse. [cite_start]We implement the **Medallion Architecture** to guarantee data quality and lineage[cite: 22].
1.  **Environment:** Setup Unity Catalog, Schemas, and Volumes.
2.  **Ingestion:** We use **Auto Loader** (`cloudFiles`) to ingest raw CSVs.
    * *Why Auto Loader?* It provides schema inference and robustness. [cite_start]Any malformed records (e.g., strings in integer columns) are captured in a special `_rescued_data` column rather than failing the pipeline[cite: 55, 56].
    * [cite_start]*Schema Evolution:* It automatically adapts if the source CSV schema changes[cite: 54].

In [0]:
# Read the parameter passed from the Job
dbutils.widgets.text("env", "dev") # Default to 'dev' if run manually
current_env = dbutils.widgets.get("env")

print(f"🚀 Running in Environment: {current_env.upper()}")

# Logic: Only Drop/Recreate tables if in Dev; in Prod, just append.
if current_env == "dev":
    print("⚠️ DEV MODE: Resetting Schema...")
    spark.sql("DROP SCHEMA IF EXISTS olist_hackathon.bronze CASCADE")
    spark.sql("CREATE SCHEMA IF NOT EXISTS olist_hackathon.bronze")
else:
    print("✅ PROD MODE: Appending to existing Schema.")

### Unity Catalog Setup (SQL)

In [0]:
%sql
-- 1. Create the Project Catalog
-- This serves as the top-level container for all our assets
CREATE CATALOG IF NOT EXISTS olist_hackathon;

-- 2. Set the active context
USE CATALOG olist_hackathon;

-- 3. Create the Medallion Layers [cite: 29]
-- Bronze: Raw ingestion (as-is)
CREATE SCHEMA IF NOT EXISTS bronze;
-- Silver: Cleaned, filtered, and enriched
CREATE SCHEMA IF NOT EXISTS silver;
-- Gold: Business-level aggregates and ML features
CREATE SCHEMA IF NOT EXISTS gold;

-- 4. Verify creation
DESCRIBE DATABASE EXTENDED bronze;

### Ingestion Configuration (Python)
We define our file paths and the mapping dictionary.

In [0]:
# --- CONFIGURATION ---
# The path where we uploaded the Kaggle CSVs
SOURCE_PATH = "/Volumes/olist_hackathon/default/raw_data/"

# Auto Loader requires a checkpoint location to track which files it has already processed
CHECKPOINT_BASE = "/Volumes/olist_hackathon/default/checkpoints/"

# Map raw filenames to clean, friendly Bronze table names 
# We map 'olist_orders_dataset.csv' -> 'orders', etc.
file_mapping = {
    "olist_orders_dataset.csv": "orders",
    "olist_order_items_dataset.csv": "order_items",
    "olist_order_reviews_dataset.csv": "reviews",
    "olist_products_dataset.csv": "products",
    "olist_customers_dataset.csv": "customers",
    "olist_sellers_dataset.csv": "sellers",
    "olist_order_payments_dataset.csv": "payments",
    "olist_geolocation_dataset.csv": "geolocation",
    "product_category_name_translation.csv": "category_translation"
}

### The Auto Loader Function (Python)
This is the core engine. It uses spark.readStream with trigger(availableNow=True) to process files efficiently.

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name

def ingest_to_bronze(source_file, table_name):
    """
    Ingests raw CSV files into a Delta Table using Databricks Auto Loader.
    
    Args:
        source_file (str): The specific CSV filename (e.g., 'olist_orders_dataset.csv')
        table_name (str): The target table name (e.g., 'orders')
    """
    source_full_path = f"{SOURCE_PATH}{source_file}"
    checkpoint_path = f"{CHECKPOINT_BASE}{table_name}"
    target_table = f"olist_hackathon.bronze.{table_name}"
    
    print(f"⏳ Starting ingestion: {source_file} -> {target_table}...")
    
    # --- AUTO LOADER DEFINITION ---
    # We use .format("cloudFiles") for Auto Loader 
    df_stream = (spark.readStream
                 .format("cloudFiles")
                 .option("cloudFiles.format", "csv")
                 # Critical: Infer types (Int, Double) instead of defaulting to Strings [cite: 60]
                 .option("cloudFiles.inferColumnTypes", "true") 
                 .option("header", "true")
                 # Critical: Location to store inferred schema for evolution [cite: 54]
                 .option("cloudFiles.schemaLocation", checkpoint_path)
                 .load(source_full_path))
    
    # --- METADATA ENRICHMENT ---
    # Adding timestamp and source filename helps with debugging and lineage
    df_final = df_stream.withColumn("ingestion_ts", current_timestamp()) \
                        .withColumn("source_file", input_file_name())
    
    # --- WRITE TO DELTA ---
    # trigger(availableNow=True) processes all available data then stops (Cost efficient)
    query = (df_final.writeStream
             .format("delta")
             .outputMode("append")
             .option("checkpointLocation", checkpoint_path)
             .option("mergeSchema", "true") # Enable schema evolution [cite: 54]
             .trigger(availableNow=True)
             .table(target_table))
    
    query.awaitTermination()
    print(f"✅ Ingestion Complete: {table_name}")

### Execution Loop (Python)
Run the function for all 9 files.

In [0]:
from pyspark.sql.functions import current_timestamp, col

# --- CONFIGURATION ---
SOURCE_PATH = "/Volumes/olist_hackathon/default/raw_data/"
CHECKPOINT_BASE = "/Volumes/olist_hackathon/default/checkpoints/"

file_mapping = {
    "olist_orders_dataset.csv": "orders",
    "olist_order_items_dataset.csv": "order_items",
    "olist_order_reviews_dataset.csv": "reviews",
    "olist_products_dataset.csv": "products",
    "olist_customers_dataset.csv": "customers",
    "olist_sellers_dataset.csv": "sellers",
    "olist_order_payments_dataset.csv": "payments",
    "olist_geolocation_dataset.csv": "geolocation",
    "product_category_name_translation.csv": "category_translation"
}

# --- CORRECTED FUNCTION ---
def ingest_to_bronze(file_name, table_name):
    """
    Ingests specific CSV files into Delta Tables using Auto Loader.
    Fix: Uses _metadata.file_path instead of input_file_name() for Unity Catalog compliance.
    """
    checkpoint_path = f"{CHECKPOINT_BASE}{table_name}"
    target_table = f"olist_hackathon.bronze.{table_name}"
    
    print(f"⏳ Starting ingestion: {file_name} -> {target_table}...")
    
    # 1. Read Stream
    df_stream = (spark.readStream
                 .format("cloudFiles")
                 .option("cloudFiles.format", "csv")
                 .option("cloudFiles.inferColumnTypes", "true") 
                 .option("header", "true")
                 .option("cloudFiles.schemaLocation", checkpoint_path)
                 .option("pathGlobFilter", file_name) # Filter for specific file
                 .load(SOURCE_PATH)) # Load from Parent Directory
    
    # 2. Metadata Enrichment (THE FIX)
    # We access the hidden '_metadata' struct to get the file path
    df_final = df_stream.withColumn("ingestion_ts", current_timestamp()) \
                        .withColumn("source_file", col("_metadata.file_path"))
    
    # 3. Write to Delta
    query = (df_final.writeStream
             .format("delta")
             .outputMode("append")
             .option("checkpointLocation", checkpoint_path)
             .option("mergeSchema", "true")
             .trigger(availableNow=True)
             .table(target_table))
    
    query.awaitTermination()
    print(f"✅ Ingestion Complete: {table_name}")

# --- EXECUTION ---
for file_name, table_name in file_mapping.items():
    try:
        ingest_to_bronze(file_name, table_name)
    except Exception as e:
        print(f"❌ Error ingesting {table_name}: {str(e)}")

print("\n🎉 All Jobs Finished.")

In [0]:
# Check one table to ensure the source_file column is populated
display(spark.sql("SELECT * FROM olist_hackathon.bronze.orders LIMIT 5"))

In [0]:
# 1. Visualize the table list
print("Tables in Bronze Schema:")
display(spark.sql("SHOW TABLES IN olist_hackathon.bronze"))

# 2. Deep Dive: Check the 'Orders' table
df_orders = spark.table("olist_hackathon.bronze.orders")
print(f"Count of Orders: {df_orders.count()}")

# 3. Quality Check: Did Auto Loader rescue any malformed data? [cite: 55]
# If this count is > 0, we have data quality issues in the source file.
rescued_count = df_orders.filter("_rescued_data IS NOT NULL").count()
print(f"Malformed Records Rescued: {rescued_count}")

display(df_orders.limit(5))